In [4]:
import cv2
import numpy as np
from keras.preprocessing import image as im
from keras import backend as k
import tensorflow as tf
from keras.models import load_model
model=load_model('finalbestmodel.hdf5')

In [5]:
# Get tags from non-filled image
def traverseCroppedSections():
    labelList = []
    for x, y, w, h in coordinates:
        croppedSection = cropImage(x, y, w, h, nonfilledimage)
        #label = tesseract.image_to_string(croppedSection)
        #labelList.append(label)
    # push these labels to database along with the coordinates and index
    
# Cropping the image
def cropImage(x, y, w, h, image):
    # Crop image here
    croppedImage = image[y:y + h,x:x + w]
    return croppedImage

# function to match template
def templateMatching(wholeimage, template):
    result = cv2.matchTemplate(wholeimage, template, cv2.TM_CCOEFF)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    return max_loc[0], max_loc[1]

# cropping and setdifferencing function
def getNonfilledCroppedSections(coordinates):
    for x, y, w, h in coordinates:
        # cropping non filled image
        nonfilledcroppedsection = cropImage(x, y, w, h, nonfilledimage)
        cv2.imshow('nonfilledcroppedsection', nonfilledcroppedsection)
        cv2.waitKey(0)
        
        #dilating 
        nonfilledcroppedsectiondilated = cropImage(x, y, w, h, nonfilledimagedilated)
        
        crop_h = nonfilledcroppedsection.shape[0]
        crop_w = nonfilledcroppedsection.shape[1]
        
        crop_x, crop_y = templateMatching(filledimage, nonfilledcroppedsection)
        
        # cropping the matched template from filled image
        filledcroppedsection = cropImage(crop_x, crop_y, crop_w, crop_h, filledimage)
        
        # subtracting nonfilled image from filled image
        setDifference = np.subtract(filledcroppedsection, nonfilledcroppedsection)
        cv2.imshow('setDifference', setDifference)
        cv2.waitKey(0)
        
        #setDifference = cv2.medianBlur(setDifference, 3)
        
        setDifference = cv2.erode(setDifference, np.ones((3, 3), np.uint8), iterations=1)
        cv2.imshow('setDifferenceeroded', setDifference)
        cv2.waitKey(0)
        
        converted_to_text = ocr(setDifference)
        print("converted_to_text")
        
        
def ocr(image):
    # detecting edges in the image
    image_edges = cv2.Canny(image, 30, 150)
    
    # dilating image to detect individual lines
    kernel_line = np.ones((10, 80), np.uint8)
    dilated_line = cv2.dilate(image_edges, kernel_line, iterations=1)
    
    # finding contours of the line
    im2, ctrs_line, hier = cv2.findContours(dilated_line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs_line = sorted(ctrs_line, key=lambda ctr: cv2.boundingRect(ctr)[1])
    
    # list of words
    word_list = []
    
    for i, ctr_line in enumerate(sorted_ctrs_line):
        # getting coordinates of the line contour
        line_x, line_y, line_w, line_h = cv2.boundingRect(ctr_line)
        
        # if condition for removing unnecessary contours
        if line_w * line_h < 10000:
            continue
        
        line = cropImage(line_x, line_y, line_w, line_h, image)
        cv2.imshow('line', line)
        cv2.waitKey(0)
        
        # detecting edges in the image
        line_edges = cv2.Canny(line, 30, 150)
        
        # dilating line to detect individual words
        kernel_word = np.ones((10, 55), np.uint8)
        dilated_word = cv2.dilate(line_edges, kernel_word, iterations=1)
        
        # finding contours of the word
        im2, ctrs_word, hier = cv2.findContours(dilated_word.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        sorted_ctrs_word = sorted(ctrs_word, key=lambda ctr: cv2.boundingRect(ctr)[0])
        
        for i, ctr_word in enumerate(sorted_ctrs_word):
            # getting coordinates of the word contour
            word_x, word_y, word_w, word_h = cv2.boundingRect(ctr_word)
            
            # if condition for removing unnecessary contours
            if word_w * word_h < 2500:
                continue
                
            word = cropImage(word_x, word_y, word_w, word_h, line)
            cv2.imshow('word', word)
            cv2.waitKey(0)
            
            # detecting edges in the image
            word_edges = cv2.Canny(word, 30, 150)
            
            # dilating word to detect individual characters
            kernel_char = np.ones((15, 15), np.uint8)
            dilated_char = cv2.dilate(word, kernel_char, iterations=1)
            
            # finding contours of the characters
            im2, ctrs_char, hier = cv2.findContours(dilated_char.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            sorted_ctrs_char = sorted(ctrs_char, key=lambda ctr: cv2.boundingRect(ctr)[0])
            
            # list of words
            character_list = []
            
            for i, ctr_char in enumerate(sorted_ctrs_char):
                # getting coordinates of the character contour
                char_x, char_y, char_w, char_h = cv2.boundingRect(ctr_char)
                
                # if condition for removing unnecessary contours
                if char_w * char_h < 180:
                    continue
                    
                charac = cropImage(char_x, char_y, char_w, char_h, word)
                cv2.imshow('charac', charac)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

In [6]:
nonfilledimage = cv2.imread('nonfilled_300.jpg',0)
nonfilledimage = cv2.threshold(nonfilledimage, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

nonfilledimagedilated = nonfilledimage.copy()
nonfilledimagedilated = cv2.dilate(nonfilledimagedilated, np.ones((2, 2), np.uint8), iterations=4)

filledimage = cv2.imread('filled_300.jpg',0)
filledimage = cv2.threshold(filledimage, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

coordinates = [[196,336,1656,119],[191,981,2137,94],[181,1149,2152,104],[920,2999,620,119],[176,1298,977,128]]

getNonfilledCroppedSections(coordinates)

cv2.destroyAllWindows()

converted_to_text
converted_to_text
converted_to_text
converted_to_text
converted_to_text
